In [30]:
from utils import read_csv_data, clean_location, build_column_vocabulary, convert_to_one_hot

In [31]:
data = read_csv_data("../data/processed_job_postings_large_noNA.csv", 
                     ["industry", "work_type", "location", "formatted_experience_level", "title_emb"], "standardized_annual_salary")
data = clean_location(data, 2)

print(len(data))
import random
random.seed(70)
random.shuffle(data)

train_data = data[:10000]
val_data = data[10000:12000]
test_data = data[12000:]

13921


In [32]:
data

[(['Luxury Goods & Jewelry',
   'FULL_TIME',
   'NY',
   'Director',
   '[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.\n 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.\n 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.\n 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.\n 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.\n 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.\n 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.\n 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.\n 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.\n 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.\n 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.\n 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.\n 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [33]:
print(type(data[1][0][4]))



<class 'str'>


In [34]:
vocab_sector = build_column_vocabulary(train_data, 0)
vocab_type = build_column_vocabulary(train_data, 1)
vocab_state = build_column_vocabulary(train_data, 2)
vocab_level = build_column_vocabulary(train_data, 3)
len(vocab_type)

7

In [35]:
vocab_state

Vocab()

In [36]:
import torch

def cat_emb(one_hot, data, emb_idx):
    results = []
    for i in range(len(data)):
        strings = data[i][0][emb_idx].replace('[', '').replace(']', '').replace('\n', '').split()
        floats = torch.tensor([float(item) for item in strings])
        tensor = torch.cat((one_hot[i], floats))
        results.append(tensor)
    return results
        

In [37]:
from mlp import MLPDataset
# temp looks like this
# [[Tensor1, Tensor2, Tensor3, Tensor4], ...]
temp = convert_to_one_hot(train_data, 
                          [(0, vocab_sector),
                           (1, vocab_type),
                           (2, vocab_state),
                           (3, vocab_level)])
temp = cat_emb(temp, train_data, 4)
train_one_hot = MLPDataset(temp, [float(target) for _, target in train_data])

temp = convert_to_one_hot(val_data, 
                          [(0, vocab_sector),
                           (1, vocab_type),
                           (2, vocab_state),
                           (3, vocab_level)])
temp = cat_emb(temp, val_data, 4)
val_one_hot = MLPDataset(temp, [float(target) for _, target in val_data])

In [38]:
train_one_hot[0][0].shape

torch.Size([659])

In [39]:
from mlp import MLP, train_model
total_features = train_one_hot[0][0].shape[0]
model = MLP(input_size=total_features, hidden_size=1000, output_size=1)  

In [40]:
train_model(model, train_one_hot, val_one_hot, 0.01, batch_size=160, num_epochs=1000)

Iter 50: Loss: nan Train mae nan, Validation mae nan
Iter 100: Loss: nan Train mae nan, Validation mae nan
Iter 150: Loss: nan Train mae nan, Validation mae nan
Iter 200: Loss: nan Train mae nan, Validation mae nan
Iter 250: Loss: nan Train mae nan, Validation mae nan
Iter 300: Loss: nan Train mae nan, Validation mae nan
Iter 350: Loss: nan Train mae nan, Validation mae nan
Iter 400: Loss: nan Train mae nan, Validation mae nan
Iter 450: Loss: nan Train mae nan, Validation mae nan
Iter 500: Loss: nan Train mae nan, Validation mae nan
Iter 550: Loss: nan Train mae nan, Validation mae nan
Iter 600: Loss: nan Train mae nan, Validation mae nan
Iter 650: Loss: nan Train mae nan, Validation mae nan


KeyboardInterrupt: 